In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [417]:
movie_train_path = '../input/train.csv'
movie_test_path = '../input/test.csv'
train = pd.read_csv(movie_train_path)
test = pd.read_csv(movie_test_path)
test_data = pd.read_csv(movie_test_path)

In [418]:
train.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,NaN,"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [419]:
test.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew
0,3001,"[{'id': 34055, 'name': 'Pokémon Collection', '...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",http://www.pokemon.com/us/movies/movie-pokemon...,tt1226251,ja,ディアルガVSパルキアVSダークライ,Ash and friends (this time accompanied by newc...,3.851534,/tnftmLMemPLduW6MRyZE0ZUD19z.jpg,NaN,"[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",7/14/07,90.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Somewhere Between Time & Space... A Legend Is ...,Pokémon: The Rise of Darkrai,"[{'id': 11451, 'name': 'pok√©mon'}, {'id': 115...","[{'cast_id': 3, 'character': 'Tonio', 'credit_...","[{'credit_id': '52fe44e7c3a368484e03d683', 'de..."
1,3002,NaN,88000,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",NaN,tt0051380,en,Attack of the 50 Foot Woman,When an abused wife grows to giant size becaus...,3.559789,/9MgBNBqlH1sG4yG2u4XkwI5CoJa.jpg,"[{'name': 'Woolner Brothers Pictures Inc.', 'i...","[{'iso_3166_1': 'US', 'name': 'United States o...",5/19/58,65.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A titanic beauty spreads a macabre wave of hor...,Attack of the 50 Foot Woman,"[{'id': 9748, 'name': 'revenge'}, {'id': 9951,...","[{'cast_id': 2, 'character': 'Nancy Fowler Arc...","[{'credit_id': '55807805c3a3685b1300060b', 'de..."
2,3003,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,tt0118556,en,Addicted to Love,Good-natured astronomer Sam is devastated when...,8.085194,/ed6nD7h9sbojSWY2qrnDcSvDFko.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",5/23/97,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A Comedy About Lost Loves And Last Laughs,Addicted to Love,"[{'id': 931, 'name': 'jealousy'}, {'id': 9673,...","[{'cast_id': 11, 'character': 'Maggie', 'credi...","[{'credit_id': '52fe4330c3a36847f8041367', 'de..."
3,3004,NaN,6800000,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",http://www.sonyclassics.com/incendies/,tt1255953,fr,Incendies,A mother's last wishes send twins Jeanne and S...,8.596012,/sEUG3qjxwHjxkzuO7plrRHhOZUH.jpg,"[{'name': 'TS Productions', 'id': 313}, {'name...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",9/4/10,130.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,The search began at the opening of their mothe...,Incendies,"[{'id': 378, 'name': 'prison'}, {'id': 539, 'n...","[{'cast_id': 6, 'character': 'Nawal', 'credit_...","[{'credit_id': '56478092c3a36826140043af', 'de..."
4,3005,NaN,2000000,"[{'id': 36, 'name': 'History'}, {'id': 99, 'na...",NaN,tt0418753,en,Inside Deep Throat,"In 1972, a seemingly typical shoestring budget...",3.217680,/n4WC3zbelz6SG7rhkWbf8m9pMHB.jpg,NaN,"[{'iso_3166_1': 'US', 'name': 'United States o...",2/11/05,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It was filmed in 6 days for 25 thousand dollar...,Inside Deep Throat,"[{'id': 279, 'name': 'usa'}, {'id': 1228, 'nam...","[{'cast_id': 1, 'character': 'Narrator (voice)...","[{'credit_id': '52fe44ce9251416c75041967', 'de..."


# EDA

In [420]:
train.describe()

,id,budget,popularity,runtime,revenue
count,3000.000000,3.000000e+03,3000.000000,2998.000000,3.000000e+03
mean,1500.500000,2.253133e+07,8.463274,107.856571,6.672585e+07
std,866.169729,3.702609e+07,12.104000,22.086434,1.375323e+08
min,1.000000,0.000000e+00,0.000001,0.000000,1.000000e+00
25%,750.750000,0.000000e+00,4.018053,94.000000,2.379808e+06
50%,1500.500000,8.000000e+06,7.374861,104.000000,1.680707e+07
75%,2250.250000,2.900000e+07,10.890983,118.000000,6.891920e+07
max,3000.000000,3.800000e+08,294.337037,338.000000,1.519558e+09


Drop id since its just an index
Every other numeric index stays

In [421]:
train.isna().sum()/len(train)

id                       0.000000
belongs_to_collection    0.798667
budget                   0.000000
genres                   0.002333
homepage                 0.684667
imdb_id                  0.000000
original_language        0.000000
original_title           0.000000
overview                 0.002667
popularity               0.000000
poster_path              0.000333
production_companies     0.052000
production_countries     0.018333
release_date             0.000000
runtime                  0.000667
spoken_languages         0.006667
status                   0.000000
tagline                  0.199000
title                    0.000000
Keywords                 0.092000
cast                     0.004333
crew                     0.005333
revenue                  0.000000
dtype: float64

In [422]:
test.isna().sum()/len(test)

id                       0.000000
belongs_to_collection    0.800591
budget                   0.000000
genres                   0.003638
homepage                 0.677126
imdb_id                  0.000000
original_language        0.000000
original_title           0.000000
overview                 0.003183
popularity               0.000000
poster_path              0.000227
production_companies     0.058663
production_countries     0.023192
release_date             0.000227
runtime                  0.000910
spoken_languages         0.009550
status                   0.000455
tagline                  0.196226
title                    0.000682
Keywords                 0.089359
cast                     0.002956
crew                     0.005002
dtype: float64

* Based on the number of missing data, outright I want to drop "belongs_to_collection"
* Drop tagline (Later down the line we can make this a feature and use the fact that the movie does not have a tagline as an input)
* Drop homepage
* Drop poster path
* Extract Country name from : production_countries
* Extract Company name from : production_countries
* Convert release_date into month and year of release
* Cast is going to be hard to get features; might be worth while extracting the top 10 actor names? Want to relate actor names to popularity and revenue; too amny features? embedding?
* Crew : same as cast
* Keywords: same as cast
* Title : embedding?
* tagline : embedding? or check even for presence maybe?
* Languages: Extract the keyword for the main language and make it a feature
* Overview: probably have to use NLP?
* Title: Can relate to popularity especially wiht sequels? 
* Genres: Multiple genres with many features, need to extract and calculate make into classes
* Budget : number
* Popularity : number


## Super Quick and Dirty

Idea here is to just use the columns which are already present and we can use with minimal cleaning.

In [423]:
# Revise wrong information

# Revising some wrong information

train.loc[train['id'] == 16,'revenue'] = 192864          # Skinning
train.loc[train['id'] == 90,'budget'] = 30000000         # Sommersby          
train.loc[train['id'] == 118,'budget'] = 60000000        # Wild Hogs
train.loc[train['id'] == 149,'budget'] = 18000000        # Beethoven
train.loc[train['id'] == 313,'revenue'] = 12000000       # The Cookout 
train.loc[train['id'] == 451,'revenue'] = 12000000       # Chasing Liberty
train.loc[train['id'] == 464,'budget'] = 20000000        # Parenthood
train.loc[train['id'] == 470,'budget'] = 13000000        # The Karate Kid, Part II
train.loc[train['id'] == 513,'budget'] = 930000          # From Prada to Nada
train.loc[train['id'] == 797,'budget'] = 8000000         # Welcome to Dongmakgol
train.loc[train['id'] == 819,'budget'] = 90000000        # Alvin and the Chipmunks: The Road Chip
train.loc[train['id'] == 850,'budget'] = 90000000        # Modern Times
train.loc[train['id'] == 1112,'budget'] = 7500000        # An Officer and a Gentleman
train.loc[train['id'] == 1131,'budget'] = 4300000        # Smokey and the Bandit   
train.loc[train['id'] == 1359,'budget'] = 10000000       # Stir Crazy 
train.loc[train['id'] == 1542,'budget'] = 1              # All at Once
train.loc[train['id'] == 1542,'budget'] = 15800000       # Crocodile Dundee II
train.loc[train['id'] == 1571,'budget'] = 4000000        # Lady and the Tramp
train.loc[train['id'] == 1714,'budget'] = 46000000       # The Recruit
train.loc[train['id'] == 1721,'budget'] = 17500000       # Cocoon
train.loc[train['id'] == 1865,'revenue'] = 25000000      # Scooby-Doo 2: Monsters Unleashed
train.loc[train['id'] == 2268,'budget'] = 17500000       # Madea Goes to Jail budget
train.loc[train['id'] == 2491,'revenue'] = 6800000       # Never Talk to Strangers
train.loc[train['id'] == 2602,'budget'] = 31000000       # Mr. Holland's Opus
train.loc[train['id'] == 2612,'budget'] = 15000000       # Field of Dreams
train.loc[train['id'] == 2696,'budget'] = 10000000       # Nurse 3-D
train.loc[train['id'] == 2801,'budget'] = 10000000       # Fracture
test.loc[test['id'] == 3889,'budget'] = 15000000       # Colossal
test.loc[test['id'] == 6733,'budget'] = 5000000        # The Big Sick
test.loc[test['id'] == 3197,'budget'] = 8000000        # High-Rise
test.loc[test['id'] == 6683,'budget'] = 50000000       # The Pink Panther 2
test.loc[test['id'] == 5704,'budget'] = 4300000        # French Connection II
test.loc[test['id'] == 6109,'budget'] = 281756         # Dogtooth
test.loc[test['id'] == 7242,'budget'] = 10000000       # Addams Family Values
test.loc[test['id'] == 7021,'budget'] = 17540562       #  Two Is a Family
test.loc[test['id'] == 5591,'budget'] = 4000000        # The Orphanage
test.loc[test['id'] == 4282,'budget'] = 20000000       # Big Top Pee-wee

In [424]:
def date_features(df):
    df['release_date'] = pd.to_datetime(df['release_date'])
    df['release_year'] = df['release_date'].dt.year
    df['release_month'] = df['release_date'].dt.month
    df['release_day'] = df['release_date'].dt.day
    df['release_quarter'] = df['release_date'].dt.quarter
    df.drop(columns=['release_date'], inplace=True)
    return df

train=date_features(train)
test=date_features(test)

train['release_year'].head(10)

0    2015
1    2004
2    2014
3    2012
4    2009
5    1987
6    2012
7    2004
8    1996
9    2003
Name: release_year, dtype: int64

In [425]:
train.columns

Index(['id', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'Keywords', 'cast', 'crew', 'revenue',
       'release_year', 'release_month', 'release_day', 'release_quarter'],
      dtype='object')

In [426]:
def drop_columns(df):
    df.drop(labels=['id', 'belongs_to_collection', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
        'poster_path', 'production_companies',
       'production_countries', 'spoken_languages',
       'tagline', 'title', 'Keywords', 'cast', 'crew'],axis=1,inplace=True)
    return df

In [427]:
train=drop_columns(train)
test=drop_columns(test)

In [428]:
from sklearn.model_selection import train_test_split
df_train,df_valid = train_test_split(train,test_size=0.2)

In [429]:
tr,va=df_train.copy(),df_valid.copy()
X_train,X_valid,y_train,y_valid = tr.drop(['revenue'],axis=1),va.drop(['revenue'],axis=1), tr['revenue'],va['revenue']

In [430]:
cat_col=[col for col in X_train.columns if X_train[col].dtype=="O"]
num_col=[col for col in X_train.columns if X_train[col].dtype in ['int64','int32','float32', 'float64']]

# Pipeline 

In [431]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [432]:
numerical_transformer = SimpleImputer(strategy='mean')
categorical_transformer= Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent')), 
                                       ('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num',numerical_transformer,num_col),
                                              ('cat',categorical_transformer, cat_col)])

In [433]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
gb = XGBRegressor(n_estimators=100,learning_rate = 0.05)
rf = RandomForestRegressor(n_estimators=100,min_samples_split=5,min_samples_leaf=3,max_features=4)
cat = CatBoostRegressor(iterations=100,learning_rate=0.05,
                                 depth=3,
                                 eval_metric='RMSE',
                                 colsample_bylevel=0.8,
                                 random_seed = 1,
                                 bagging_temperature = 0.2,
                                 metric_period = None,
                                 early_stopping_rounds=200
                                )

In [434]:
xgb_pl = Pipeline(steps=[('preprocessor', preprocessor),('xgb',gb)])
rf_pl = Pipeline(steps=[('preprocessor', preprocessor),('randomforest',rf)])
cat_pl = Pipeline(steps=[('preprocessor', preprocessor),('catBoost',cat)])


In [435]:
xgb_pl.fit(X_train,y_train)
rf_pl.fit(X_train,y_train)
cat_pl.fit(X_train,y_train)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/conda/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


0:	learn: 150172880.8903933	total: 3.36ms	remaining: 333ms
1:	learn: 146337691.2907563	total: 6.75ms	remaining: 331ms
2:	learn: 142311480.2325180	total: 10ms	remaining: 323ms
3:	learn: 138507316.8195122	total: 14.2ms	remaining: 342ms
4:	learn: 134942342.2421011	total: 17ms	remaining: 323ms
5:	learn: 131884076.6898670	total: 20.1ms	remaining: 315ms
6:	learn: 128647863.9569782	total: 23.6ms	remaining: 314ms
7:	learn: 125797161.3256752	total: 26.8ms	remaining: 308ms
8:	learn: 122873724.8788623	total: 29.7ms	remaining: 301ms
9:	learn: 120349590.6168873	total: 32.9ms	remaining: 296ms
10:	learn: 118278879.3959068	total: 35.9ms	remaining: 290ms
11:	learn: 116090813.1005381	total: 38.9ms	remaining: 285ms
12:	learn: 113809629.6861348	total: 41.9ms	remaining: 280ms
13:	learn: 111689610.8623388	total: 44.8ms	remaining: 275ms
14:	learn: 109885243.1855489	total: 47.7ms	remaining: 271ms
15:	learn: 108158053.8496190	total: 51ms	remaining: 268ms
16:	learn: 106708152.6012527	total: 54ms	remaining: 264m

Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
       verbose=0), ['budget', 'popularity', 'runtime', 'rele...=True))]), ['status'])])), ('catBoost', <catboost.core.CatBoostRegressor object at 0x7f80698bb0b8>)])

In [436]:
xgb_pred = xgb_pl.predict(X_valid)
rf_pred = rf_pl.predict(X_valid)
cat_pred = cat_pl.predict(X_valid)

In [437]:
from sklearn.metrics import mean_absolute_error
mae_rf = mean_absolute_error(rf_pred,y_valid)
mae_xgb =  mean_absolute_error(xgb_pred,y_valid)
mae_cat= mean_absolute_error(cat_pred,y_valid)
print('MAE RF: ', mae_rf, 'MAE XGB: ', mae_xgb,'MAE CAT: ', mae_cat )

MAE RF:  38468054.803794235 MAE XGB:  38667460.45166667 MAE CAT:  39377967.35591674


In [438]:
np.mean(np.abs((rf_pred - y_valid) / rf_pred)) * 100, np.mean(np.abs((xgb_pred - y_valid) / xgb_pred)) * 100,np.mean(np.abs((cat_pred - y_valid) / cat_pred)) * 100

(89.03796043251315, 81.10605034059567, 83.42405696015624)

In [439]:
pred= xgb_pl.predict(test)

In [440]:
submission = pd.DataFrame({'id':test_data['id'],
                          'revenue':pred})

In [442]:
submission.to_csv('submission.csv',index=False)